In [0]:
#from google.colab.patches import cv2_imshow
import numpy as np
import cv2 as cv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import random, os, time


import ai, map

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import sys
print(sys.version)
!python -V

print(device)

import cv2 as cv
print(cv.__version__)

print(torch.__version__)

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
Python 3.6.9
cuda
4.1.2
1.5.0+cu101


In [0]:
car_file = 'car.jpg'
city_file = 'citymap.png'
city_map_file = "MASK1.png"
car_img = cv.imread(car_file)
car1 = map.car(0,0,0)
city1 = map.city(city_file)
citymap1 = map.city(city_map_file)

**Function to evaluates the policy by calculating its average reward over 10 episodes**


In [0]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

def evaluate_policy(policy, eval_episodes=5):
  avg_reward = 0
  for count in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(np.array(obs))
      obs, reward, done = env.step(action)
      avg_reward += reward
      #print("avg_reward = ", avg_reward)
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Average Reward over the Evaluation Step: %f" % (avg_reward))
  print ("---------------------------------------")
  return avg_reward

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pickle

rb_file = "/content/drive/My Drive/EVA_Endgame_Model/rb.pickle"

def save_rb(replay_buffer, rb_file, iteration):

    with open(rb_file, 'wb') as handle:
        pickle.dump(replay_buffer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    f = open("/content/drive/My Drive/EVA_Endgame_Model/iteration_count.txt", "w")
    f.write(str(iteration))
    f.close()

def load_rb(rb_file):    

    f = open("/content/drive/My Drive/EVA_Endgame_Model/iteration_count.txt", "rt")
    #print(f.read())
    count = f.read()

    with open(rb_file, 'rb') as handle:
        replay_buffer1 = pickle.load(handle)
        
    return replay_buffer1, count

def update_stats(total_timesteps, episode_num, episode_reward, done, on_road_count, off_road_count):

    f = open("/content/drive/My Drive/EVA_Endgame_Model/stats.csv", "a")
    text = str(total_timesteps) + ", " 
    text = text + str(episode_num) + ", "
    text = text + str(episode_reward) + ", "
    text = text + str(done) + ", "
    text = text + str(on_road_count) + ", "
    text = text + str(off_road_count) + "\n"
    f.write(text)
    f.close()



In [7]:
env = map.env(car1, city1, citymap1, car_img) # Instantiate the environment
seed = 8 # Random seed number
start_timesteps = 1e4 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
eval_freq = 5e3 # How often the evaluation step is performed (after how many timesteps)
max_timesteps = 5e5 # Total number of iterations/timesteps
save_models = False # Boolean checker whether or not to save the pre-trained model
expl_noise = 0.1 # Exploration noise - STD value of exploration Gaussian noise
batch_size = 100 # Size of the batch
discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
tau = 0.005 # Target network update rate
policy_noise = 0.2 # STD of Gaussian noise added to the actions for the exploration purposes
noise_clip = 0.5 # Maximum value of the Gaussian noise added to the actions (policy)
policy_freq = 2 # Number of iterations to wait before the policy network (Actor model) is updated
episode_reward = 0
episode_timesteps = 0


file_name = "%s_%s" % ("TD3_car", str(seed))
print ("---------------------------------------")
print ("Settings: %s" % (file_name))
print ("---------------------------------------")
if not os.path.exists("./results"):
  os.makedirs("./results")
if save_models and not os.path.exists("./pytorch_models"):
  os.makedirs("./pytorch_models")

torch.manual_seed(seed)
np.random.seed(seed)
state_dim = env.state_dim
action_dim = env.action_dim
max_action = env.max_action

---------------------------------------
Settings: TD3_car_8
---------------------------------------


In [8]:
env.reset()
action = env.sample_action()
new_obs, reward, done = env.step(action)
#print(type(new_obs), reward, done)

policy = ai.TD3(state_dim, action_dim, max_action)
replay_buffer = ai.ReplayBuffer()
evaluations = [evaluate_policy(policy)]

work_dir = mkdir('exp', 'brs')
monitor_dir = mkdir(work_dir, 'monitor')
max_episode_steps = env._max_episode_steps

total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True
t0 = time.time()

Step Function :: Boundary hit 50 times at location::463.95::635.00 On Road Steps Percentage: 34.77
Step Function :: Boundary hit 50 times at location::401.83::635.00 On Road Steps Percentage: 31.33
Step Function :: Boundary hit 50 times at location::433.96::635.00 On Road Steps Percentage: 37.95
Step Function :: Boundary hit 50 times at location::412.60::635.00 On Road Steps Percentage: 32.84
Step Function :: Boundary hit 50 times at location::449.45::635.00 On Road Steps Percentage: 42.59
---------------------------------------
Average Reward over the Evaluation Step: -1690.400000
---------------------------------------


In [0]:
episode_reward = 0
f = open("/content/drive/My Drive/EVA_Endgame_Model/stats.csv", "w")
f.write("total_timesteps, episode_num, episode_reward, done, on_road_count, off_road_count\n")
f.close()
re_train_flag = False

if (re_train_flag):
  policy.load(file_name, "/content/drive/My Drive/EVA_Endgame_Model")
  replay_buffer, iter_count = load_rb(rb_file)
  total_timesteps = int(iter_count)

In [10]:
while total_timesteps < max_timesteps:
  
  # If the episode is done
  if done:

    # If we are not at the very beginning, we start the training process of the model
    if (total_timesteps != 0):
      print("Total Timesteps: {} Episode Num: {} Reward: {} episode_timesteps: {}  On Road Step Percentage: {:.2f}".format(total_timesteps, episode_num, episode_reward, episode_timesteps, (env.episode_onroad*100/(episode_timesteps+1))))
      policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

    # We evaluate the episode and we save the policy
    if timesteps_since_eval >= eval_freq:
      timesteps_since_eval %= eval_freq
      evaluations.append(evaluate_policy(policy))
      file_name = "%s_%s_%s" % ("TD3_car", str(seed), str(total_timesteps))
      if save_models: policy.save(file_name, directory="./pytorch_models")
      policy.save(file_name, directory="/content/drive/My Drive/EVA_Endgame_Model")
      #save_rb(replay_buffer, rb_file, total_timesteps)
      np.save("./results/%s" % (file_name), evaluations)
    
    # When the training step is done, we reset the state of the environment
    obs = env.reset()
    
    # Set the Done to False
    done = False
    
    # Set rewards and episode timesteps to zero
    episode_reward = 0
    episode_timesteps = 0
    episode_num += 1
  
  # Before 10000 timesteps, we play random actions
  if total_timesteps < start_timesteps:
    action = env.sample_action()
  else: # After 10000 timesteps, we switch to the model  
      action = policy.select_action(obs)
      # If the explore_noise parameter is not 0, we add noise to the action and we clip it
      if expl_noise != 0:
          action = (action + np.random.normal(0, expl_noise, size=1)).clip(-max_action, max_action)[0]
  
  # The agent performs the action in the environment, then reaches the next state and receives the reward
  new_obs, reward, done = env.step(action)
  
  # We check if the episode is done
  done_bool = 0 if episode_timesteps + 1 == env._max_episode_steps else float(done)
  
  # We increase the total reward
  episode_reward += reward
  
  # We store the new transition into the Experience Replay memory (ReplayBuffer)
  replay_buffer.add((obs, new_obs, action, reward, done_bool))
  update_stats(total_timesteps, episode_num, episode_reward, done, env.on_road_count, env.off_road_count)

  # We update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
  obs = new_obs
  episode_timesteps += 1
  total_timesteps += 1
  timesteps_since_eval += 1

# We add the last policy evaluation to our list of evaluations and we save our model
evaluations.append(evaluate_policy(policy))
if save_models: policy.save("%s" % (file_name), directory="./pytorch_models")
np.save("./results/%s" % (file_name), evaluations)

Step Function :: Completed steps limit at location::813.70::384.06 On Road Steps Percentage: 19.81
Total Timesteps: 5001 Episode Num: 1 Reward: -20266.0 episode_timesteps: 5001  On Road Step Percentage: 19.81
Step Function :: Completed steps limit at location::118.56::601.08 On Road Steps Percentage: 7.98
Step Function :: Completed steps limit at location::126.07::583.45 On Road Steps Percentage: 6.68
Step Function :: Completed steps limit at location::148.64::599.23 On Road Steps Percentage: 8.04
Step Function :: Completed steps limit at location::125.67::599.41 On Road Steps Percentage: 7.46
Step Function :: Completed steps limit at location::102.66::594.30 On Road Steps Percentage: 7.66
---------------------------------------
Average Reward over the Evaluation Step: -23378.000000
---------------------------------------
Step Function :: Boundary hit 50 times at location::25.00::409.37 On Road Steps Percentage: 11.16
Total Timesteps: 5430 Episode Num: 2 Reward: -2524.5 episode_timeste

KeyboardInterrupt: ignored